<a href="https://colab.research.google.com/github/KhoaCoderz/fake-new-detection/blob/main/FakeNewDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas numpy scikit-learn tensorflow transformers

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, create_optimizer
import tensorflow as tf
import re
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Đọc file csv
df = pd.read_csv("/content/drive/MyDrive/Fake_New_Detection/public_train.csv")

# Chỉ lấy nội dung và nhãn
df = df[['post_message', 'label']].dropna()

# Đổi tên cột cho dễ dùng
df = df.rename(columns={'post_message': 'text', 'label': 'label'})

print(df.head())
print(df['label'].value_counts())


                                                text  label
0  THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...      0
1                                              <URL>      0
2  TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...      0
3  Cơ quan Cạnh tranh và Thị trường Anh quyết địn...      0
4  Thêm 7 ca tại Quảng Nam liên quan đến hành khá...      0
label
0    3637
1     734
Name: count, dtype: int64


In [6]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-ZÀ-ỹ\s]', ' ', text)  # giữ chữ cái tiếng Việt, bỏ số/ký tự
    text = re.sub(r'\s+', ' ', text).strip()     # bỏ khoảng trắng thừa
    return text

df['clean_text'] = df['text'].apply(preprocess_text)


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# PhoBERT base (pretrained trên tiếng Việt)
model_name = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Chia train/test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

# Tạo dataset HuggingFace
train_df = pd.DataFrame({"text": train_texts, "label": train_labels})
test_df  = pd.DataFrame({"text": test_texts, "label": test_labels})

dataset = Dataset.from_pandas(pd.concat([train_df, test_df]))
dataset = dataset.train_test_split(test_size=0.2)

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3496 [00:00<?, ? examples/s]

Map:   0%|          | 0/875 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # giảm từ 16 xuống 8
    per_device_eval_batch_size=8,
    num_train_epochs=1,              # train 1 epoch thôi để test nhanh
    weight_decay=0.01,
    fp16=True,                       # mixed precision (nếu GPU hỗ trợ)
    report_to="none"                 # tránh log ra wandb
)

for param in model.base_model.parameters():
    param.requires_grad = False

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-2283037923.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.416300,0.418982


TrainOutput(global_step=437, training_loss=0.42442493526013547, metrics={'train_runtime': 4581.2228, 'train_samples_per_second': 0.763, 'train_steps_per_second': 0.095, 'total_flos': 459918124769280.0, 'train_loss': 0.42442493526013547, 'epoch': 1.0})

In [11]:
save_dir = "./fake_news_phobert"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('./fake_news_phobert/tokenizer_config.json',
 './fake_news_phobert/special_tokens_map.json',
 './fake_news_phobert/vocab.txt',
 './fake_news_phobert/bpe.codes',
 './fake_news_phobert/added_tokens.json')

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

save_dir = "./fake_news_phobert"

model = AutoModelForSequenceClassification.from_pretrained(save_dir)
tokenizer = AutoTokenizer.from_pretrained(save_dir)

fake_news_clf = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0  # GPU nếu có
)

# Test dự đoán
text = "Chính phủ vừa công bố chính sách mới hỗ trợ doanh nghiệp."
result = fake_news_clf(text)[0]

print(result)


Device set to use cpu


{'label': 'LABEL_0', 'score': 0.8994495272636414}


In [14]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.2 MB/s eta 0:00:00


In [35]:
!pip install pyngrok
from pyngrok import ngrok

# Dán token của bạn
!ngrok config add-authtoken "32VNA8IIBdQjJVXzLaf3aM23e1u_6rJvjCio99Gy4zDHz6t3X"

# Chạy streamlit
!streamlit run app.py --server.port 8501 &>/dev/null&

# Tạo tunnel
public_url = ngrok.connect(8501)
public_url

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


<NgrokTunnel: "https://de7b7ca9a60d.ngrok-free.app" -> "http://localhost:8501">